In [2]:
import torch
import pandas as pd

In [ ]:
def kl_divergence(solution: pd.DataFrame, submission: pd.DataFrame, epsilon: float):
    # Overwrite solution for convenience
    for col in solution.columns:
        # Clip both the min and max following Kaggle conventions for related metrics like log loss
        # Clipping the max avoids cases where the loss would be infinite or undefined, clipping the min
        # prevents users from playing games with the 20th decimal place of predictions.
        submission[col] = np.clip(submission[col], epsilon, 1 - epsilon)

        y_nonzero_indices = solution[col] != 0
        solution[col] = solution[col].astype(float)
        solution.loc[y_nonzero_indices, col] = solution.loc[y_nonzero_indices, col] * np.log(solution.loc[y_nonzero_indices, col] / submission.loc[y_nonzero_indices, col])
        # Set the loss equal to zero where y_true equals zero following the scipy convention:
        # https://docs.scipy.org/doc/scipy/reference/generated/scipy.special.rel_entr.html#scipy.special.rel_entr
        solution.loc[~y_nonzero_indices, col] = 0

    if micro_average:
        return np.average(solution.sum(axis=1), weights=sample_weights)
    else:
        return np.average(solution.mean())

In [9]:
actual = torch.randint(0, 10, (10, 6))
actual = actual/actual.sum(dim=1, keepdim=True)
predict = actual + torch.randn(10, 6)
log_predict = predict.float().log_softmax(dim=-1)
actual, log_predict

(tensor([[0.0909, 0.3182, 0.0455, 0.2727, 0.1364, 0.1364],
         [0.2812, 0.1250, 0.1875, 0.2812, 0.1250, 0.0000],
         [0.1429, 0.0000, 0.1429, 0.4286, 0.2143, 0.0714],
         [0.0400, 0.1600, 0.2000, 0.2400, 0.1600, 0.2000],
         [0.0435, 0.2609, 0.0435, 0.3478, 0.2609, 0.0435],
         [0.2195, 0.0244, 0.1707, 0.1951, 0.1707, 0.2195],
         [0.1562, 0.1875, 0.2812, 0.1875, 0.1562, 0.0312],
         [0.0000, 0.2333, 0.3000, 0.3000, 0.0000, 0.1667],
         [0.4211, 0.0526, 0.2105, 0.1053, 0.1053, 0.1053],
         [0.2500, 0.1250, 0.1562, 0.0938, 0.1562, 0.2188]]),
 tensor([[-2.2240, -1.3391, -1.7997, -1.1525, -2.3861, -2.8724],
         [-0.8621, -0.8785, -4.8222, -3.4024, -2.6319, -3.0150],
         [-1.2314, -2.7200, -1.1352, -1.7841, -2.4149, -2.7557],
         [-1.6503, -2.4486, -1.9649, -0.8084, -2.5924, -2.7969],
         [-2.6310, -2.9300, -0.6184, -1.8352, -2.4931, -2.3691],
         [-1.6516, -0.7760, -1.4824, -3.4889, -3.4864, -2.8172],
         [-1.9847,

In [ ]:
solution.loc[y_nonzero_indices, col] * np.log(solution.loc[y_nonzero_indices, col] / submission.loc[y_nonzero_indices, col])

In [18]:
predict = torch.clip(predict, min=10**-16, max=1-10**-16)

In [24]:
actual[1], (predict+1e-6)[1]

(tensor([0.2812, 0.1250, 0.1875, 0.2812, 0.1250, 0.0000]),
 tensor([1.0000e+00, 1.0000e+00, 1.0000e-06, 1.0000e-06, 1.0000e-06, 1.0000e-06]))

In [33]:
l = (actual * torch.log(actual/(predict+1e-6))).reshape(-1)

In [41]:
mask = torch.isnan(l)

In [44]:
l[~mask].mean()

tensor(1.0595)